<a href="https://colab.research.google.com/github/nmuzyka2004/test/blob/main/%D0%A0%D0%B5%D1%88%D0%B5%D0%BD%D0%B8%D0%B5_%D0%B4%D0%BE%D0%BC%D0%B0%D1%88%D0%BA%D0%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Домашка

1. Загрузить файл https://gist.githubusercontent.com/andron23/57efe40e7ef578d01c2f48497eea29bf/raw/d89cd282d0c6350246799db6b402d3391541d0a1/sales.csv
2. Добавить информацию о сумме продаж к исходному датафрейму
3. Сделать агрегацию и посчитать метрику ARPU в разрезе рекламных каналов
4. Сделать новые управленческие выводы на основании новой информации

При решении домашнего задания за основу берем код с лайв-интенсива: https://colab.research.google.com/drive/1Qh94MRmcHFdxOiQJxYAifzLBQIw17ONX?usp=sharing#scrollTo=ykQXs3sMkG_B

По аналогии с тем, как делали это раньше, скачиваем файл

In [ ]:
!wget https://gist.githubusercontent.com/andron23/57efe40e7ef578d01c2f48497eea29bf/raw/d89cd282d0c6350246799db6b402d3391541d0a1/sales.csv

Далее загружаем данные в новый датафрейм

In [ ]:
import pandas as pd

sales = pd.read_csv('sales.csv', decimal='.', sep=';')

Смотрим на его внутреннюю структуру

In [ ]:
sales

Нам нужно соединить две таблицы - исходный датафрейм `df` и новый `sales`. Если вы знакомы с SQL, то нам нужно сделать аналог операции `JOIN`. Если знакомы с Excel - то `ВПР`.

В Pandas это делается с помощью метода `merge`.

У нас 2 общих столбца в каждом датафрейме - `date` и `placement`. Именно по ним нужно соединять. Это указывается с помощью аргументов `left_on` и `right_on`.

Также мы указываем аргумент `how='outer'`, чтобы оставить не только те строки, в которых найдено соответствие (т.е. только дни, в которые были продажи), а вообще все строки. Если в какой-то день продажи не было, то в итоговой таблице там будет `NaN`.

In [ ]:
df = df.merge(sales, left_on=['date', 'placement'], right_on=['date', 'placement'], how='outer')

Снова делаем агрегирование, добавляя на этот раз сумму покупок в столбец `sales`.

In [ ]:
df2 = df.groupby('placement').agg({
    'clicks': [
        ('sum_clicks', 'sum'),
        ('mean_clicks', 'mean')
    ],
    'impressions': [
       ('sum_impressions', 'sum'),
       ('mean_impressions', 'mean')
    ],
    'spent': [
        ('spent_daily', 'mean'),
        ('spent_total', 'sum')
    ],
    'leads': [('cnt_leads', 'sum')],
    'qualleads': [('cnt_qual', 'sum')],
    'customers': [('customers_sum', 'sum')],
    'rubs': [('sales', 'sum')]
}).reset_index()

df2.columns = df2.columns.droplevel(0)

ARPU - average revenue per user, средний доход с покупателя. В нашем случае это средний чек.

Считаем `ARPU` как отношение суммы покупок к количеству покупателей.

In [ ]:
df2['mean_crq'] = df2['cnt_qual'] / df2['cnt_leads']
df2['mean_cpql'] = df2['spent_total'] / df2['cnt_qual']
df2['mean_cpl'] = df2['spent_total'] / df2['cnt_leads']
df2['mean_cr1'] = df2['cnt_leads'] / df2['sum_clicks']
df2['mean_cpc'] = df2['spent_total'] / df2['sum_clicks']
df2['cac'] = df2['spent_total'] / df2['customers_sum']
df2['cr2'] = df2['customers_sum'] / df2['cnt_leads']

df2['arpu'] = df2['sales'] / df2['customers_sum']

Смотрим на результат

In [ ]:
df2

Ранее мы сделали вывод, что хоть ВК и работает хуже, там намного ниже стоимость клиента.

Однако теперь мы наблюдаем обратное - оказывается, в ВК мы с клиента зарабатываем очень мало, в то время как с Инстаграма - в 3 раза больше (хоть клиентов там и меньше).

Получается, нам снова нужно поменять наше управленческое решение - теперь давайте точно останавливать ВК и реинвестировать в Инстаграм! 😱

Однако, перед этим давайте посчитаем - а в итоге-то какой рекламный канал приносит нам больше денег. Посчитаем прибыль как разность заработанных и потраченных денег.

In [ ]:
df2['profit'] = df2['sales'] - df2['spent_total']

In [ ]:
df2

И наблюдаем забавную ситуацию - оба канала пока работают в минус 😁 Получается, нужно отключить оба канала.

На самом деле, ситуация очень непростая, но по факту вполне себе реальная - редко в жизни встречаются однозначные ответы.

Мы предлагаем на основании проведенного анализа такие рекомендации:

1. Нужно увеличить CR1 в Инстаграме - это позволит снизить стоимость лида, а значит и CAC при тех же конверсиях. Например, изменить текст рекламных объявлений - судя по всему, у людей не сходится ожидание/реальность и они не оставляют заявки на сайте.
2. Нужно снизить CPC в ВК - это также повлияет на снижение CAC. Например, можно попробовать более яркие объявления или другую аудиторию.
3. Бюджет пока можно нигде не увеличивать - слишком все пока шатко.
4. Нужно увеличить CR2 в Инстаграме - за счет этого CAC будет сильно меньше. Для этого внутри воронки нужно усилить прогрев - людям недостаточно мотивации покупать. Также стоит провести кастдев (исследование) людей из Инстаграма - раз на той же воронке люди из ВК покупают нормально, видимо, что-то не так с аудиторией из Инстаграма. Иногда дело может быть в мелочах - люди могут просто не знать, что вы принимаете оплаты из-за рубежа.
5. Нужно увеличить ARPU в ВК - за счет этого прибыль с клиента будет больше. Почему-то люди из ВК не видят для себя ценности покупать более дорогой тариф. Нужно отредактировать воронку - добавить туда сообщений, которые убеждают купить более дорогой тариф. Также можно пообщаться с уже купившими - почему они выбрали дешевый тариф, может быть что-то станет понятно.